## BigQuery ML Tutorial

## 1 Ingest data into BigQuery 
Here, we use the natality dataset which is already available in BigQuery [here](https://bigquery.cloud.google.com/table/publicdata:samples.natality?tab=details)

## 2 Explore the data

In [ ]:
SELECT
  *
FROM
  publicdata.samples.natality
WHERE
  year > 1998
  AND plurality > 0
  AND mother_age > 0
  AND gestation_weeks > 0
  AND weight_pounds > 0
  AND weight_gain_pounds > 0
LIMIT 15

## 3 Define and engineer the features

In [ ]:
SELECT
  weight_pounds, -- target of prediction (label)
  CAST(is_male AS STRING) AS is_male,
  CAST(plurality AS STRING) AS plurality,
  mother_age,
  gestation_weeks,
  weight_gain_pounds,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE
  year > 1998
  AND plurality > 0
  AND mother_age > 0
  AND gestation_weeks > 0
  AND weight_pounds > 0
  AND weight_gain_pounds > 0
LIMIT 15 

## 4 Build and train the model

In [ ]:
CREATE MODEL BigQDemo.NewBornWeight_Model
OPTIONS
  (model_type='linear_reg', labels=['weight_pounds']) AS
  
WITH natality_dset AS -- code from previous step
(
SELECT
  weight_pounds,
  CAST(is_male AS STRING) AS is_male,
  CAST(plurality AS STRING) AS plurality,
  mother_age,
  gestation_weeks,
  weight_gain_pounds,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE
  year > 1998
  AND plurality > 0
  AND mother_age > 0
  AND gestation_weeks > 0
  AND weight_pounds > 0
  AND weight_gain_pounds > 0
)
SELECT
    weight_pounds,
    is_male,
    plurality,
    mother_age,
    gestation_weeks,
    weight_gain_pounds
FROM
    natality_dset
WHERE
  MOD(hashmonth, 5) < 4  -- taking 80% of the data as training data 

## 5 Evaluate the trained model

In [ ]:
SELECT 
  *
FROM
  ml.EVALUATE(MODEL BigQDemo.NewBornWeight_Model,
  (
  SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    CAST(plurality AS STRING) AS plurality,
    mother_age,
    gestation_weeks,
    weight_gain_pounds
  FROM
    publicdata.samples.natality
  WHERE
    year > 1998
    AND plurality > 0
    AND mother_age > 0
    AND gestation_weeks > 0
    AND weight_pounds > 0
    AND weight_gain_pounds > 0
    AND MOD (ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))),5) >= 4
    
    ))

## 6 Predict with the model

In [ ]:
SELECT 
  *
FROM
  ml.PREDICT(MODEL BigQDemo.NewBornWeight_Model,
  (
  SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    CAST(plurality AS STRING) AS plurality,
    mother_age,
    gestation_weeks,
    weight_gain_pounds
  FROM
    publicdata.samples.natality
  WHERE
    year > 1998
    AND plurality > 0
    AND mother_age > 0
    AND gestation_weeks > 0
    AND weight_pounds > 0
    AND weight_gain_pounds > 0
    ))
LIMIT 50
 